# 모델 2 - 기능 3에 대해 추가 처리 가능한 모델2 작성 (1)
#### 모델 2는 모델 1에서 기능 3 지하철 트리거 인식 시 작동되는 모델임
#### 모델 1에서 기능 3으로 분류(카테고리 :3) 인 경우 해당 모델에서 역별 분류 작업 수행
#### 최종 수정일 : 20-05-27
* * *

## RNN 모델

* * *
## 2. 모델링
### 2.1 데이터 구성

> [변경사항]

* * *
> [기존과 동일]
- model_1.npz 불러오기

In [1]:
import glob
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# 다운그레이드,,

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
# %watermark -v -m -p scipy,numpy,sklearn,tensorflow

In [4]:
# 알림, 차량 엔진, 차량 경적, 지하철 트리거 순
sound_data = np.load('data.npz')
X_data = sound_data['X']
y_data = sound_data['y']
X_data.shape, y_data.shape

((129, 193), (129, 1))

In [5]:
from sklearn.model_selection import train_test_split
X_sub, X_test, y_sub, y_test = train_test_split(X_data, y_data, test_size=0.1)
X_train, X_val, y_train, y_val = train_test_split(X_sub, y_sub, test_size=0.1)
len(X_train), len(X_val), len(X_test), len(y_train), len(y_val), len(y_test)

(104, 12, 13, 104, 12, 13)

In [6]:
X_train.shape, y_train.shape

((104, 193), (104, 1))

In [7]:
training_epochs = 6000
n_dim = 193
n_classes = 1
n_hidden_units_one = 300
n_hidden_units_two = 200
n_hidden_units_three = 100
learning_rate = 0.1
sd = 1 / np.sqrt(n_dim)

In [8]:
X = tf.placeholder(tf.float32,[None,n_dim])
Y = tf.placeholder(tf.float32,[None,n_classes])

W_1 = tf.Variable(tf.random_normal([n_dim, n_hidden_units_one], mean=0, stddev=sd), name="w1")
b_1 = tf.Variable(tf.random_normal([n_hidden_units_one], mean=0, stddev=sd), name="b1")
h_1 = tf.nn.sigmoid(tf.matmul(X, W_1) + b_1)

W_2 = tf.Variable(tf.random_normal([n_hidden_units_one, n_hidden_units_two], mean=0, stddev=sd), name="w2")
b_2 = tf.Variable(tf.random_normal([n_hidden_units_two], mean=0, stddev=sd), name="b2")
h_2 = tf.nn.tanh(tf.matmul(h_1, W_2) + b_2)

W_3 = tf.Variable(tf.random_normal([n_hidden_units_two, n_hidden_units_three], mean=0, stddev=sd), name="w3")
b_3 = tf.Variable(tf.random_normal([n_hidden_units_three], mean=0, stddev=sd), name="b3")
h_3 = tf.nn.sigmoid(tf.matmul(h_2, W_3) + b_3)

W = tf.Variable(tf.random_normal([n_hidden_units_three, n_classes], mean=0, stddev=sd), name="w")
b = tf.Variable(tf.random_normal([n_classes], mean = 0, stddev=sd), name="b")
y_ = tf.nn.softmax(tf.matmul(h_3, W) + b)

init = tf.initialize_all_variables()

cost_function = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(y_), reduction_indices=[1])) 
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

saver = tf.train.Saver()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [11]:
cost_history = np.empty(shape=[1],dtype=float)
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        _,cost = sess.run([optimizer, cost_function], feed_dict={X: X_sub, Y: y_sub})
        cost_history = np.append(cost_history,cost)
    
    print('Validation accuracy: ',round(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}) , 3))
    saver.save(sess, "model_2.ckpt")

Validation accuracy:  1.0


* * *
## 2. 모델링
### 2.3 모델 저장

> [변경사항]
- pkl, json, pb, tflite로 저장
* * *
> [기존과 동일]


In [10]:
# 모델 pkl로 저장하기
import joblib
joblib.dump(model, 'model_2.pkl')

NameError: name 'model' is not defined